In [1]:
import os

In [2]:
# move to project root
while True:
    # get list of directories
    dirs = os.listdir()
    if "README.md" in dirs:
        break
    else:
        os.chdir("..")
print(os.getcwd())

/home/ra/Codes/multilang_timescale


In [54]:
import time
import logging

logging.basicConfig(level=logging.DEBUG)

import numpy as np
import pandas as pd

from scipy.stats import zscore

from sklearn.linear_model import RidgeCV, Ridge
from sklearn.model_selection import KFold

from matplotlib.pyplot import figure, cm

from src.vm_tutorial_sklearn.stimulus_utils import (
    load_grids_for_stories,
    load_generic_trfiles,
    load_story_info,
)
from src.vm_tutorial_sklearn.dsutils import make_word_ds, make_phoneme_ds
from src.vm_tutorial_sklearn.util import make_delayed, load_dict
from src.vm_tutorial_sklearn.hard_coded_things import (
    test_stories,
    train_stories,
    silence_length,
    noise_trim_length,
)

from src.config import (
    grids_en_path,
    trs_en_path,
    feature_sets_en_path,
    reading_data_en_path,
)

# Parameters

In [4]:
subject = "07"
timescale = "2_4_words"

# timescale = "2_4_words"
# data_dir = os.path.join(reading_data_path, "data_en")
# featureset_dir = os.path.join(reading_data_path, "featureset_en")
# grid_dir = os.path.join(reading_data_path, "grids_en")
# trf_dir = os.path.join(reading_data_path, "trfiles_en")

# Loading Stimuli

In [5]:
# adding audio
all_stories = train_stories + test_stories

In [6]:
# load story info
story_info = load_story_info(
    story_name=train_stories[0], grids_path=grids_en_path, trs_path=trs_en_path
)

# Loading feature set

In [7]:
bert_feature_set = os.path.join(feature_sets_en_path, "timescales_BERT_all.npz")
mbert_feature_set = os.path.join(feature_sets_en_path, "timescales_mBERT_all.npz")

bert_features_meta = os.path.join(
    feature_sets_en_path, "timescales_BERT_all_train_meta.csv"
)
mbert_features_meta = os.path.join(
    feature_sets_en_path, "timescales_mBERT_all_train_meta.csv"
)

In [8]:
bert_features = np.load(bert_feature_set, allow_pickle=True)

bert_train_feature = bert_features["train"].tolist()
bert_test_feature = bert_features["test"].tolist()

In [9]:
# bert_train_feature_fast = bert_train_feature[timescala]
# bert_test_feature_fast = bert_test_feature[timescale]

In [10]:
# bert_meta = pd.read_csv(bert_features_meta, index_col=0)
# bert_meta_fast = bert_meta[bert_meta["timescale_name"] == timescale]

# bert_meta_fast.sort_values("index", inplace=True)
# # now get rolling sum of feature_len
# feature_end_idx = bert_meta_fast["feature_len"].rolling(min_periods=1, window=10).sum()
# feature_end_idx = feature_end_idx.astype(int)
# feature_end_idx = feature_end_idx.tolist()

# # zip with story_names
# end_index = {a: b for a, b in zip(bert_meta_fast["story_name"], feature_end_idx)}

## Delaying Feature Set

In [11]:
bert_train_feature.keys()

dict_keys(['2_4_words', '4_8_words', '8_16_words', '16_32_words', '32_64_words', '64_128_words', '128_256_words', '256+ words'])

In [12]:
ndelays = 4
delays = np.arange(1, ndelays + 1)

# delaying all features
delayed_train_features = {}
delayed_test_features = {}

for story in bert_train_feature.keys():
    delayed_train_features[story] = make_delayed(
        bert_train_feature[story], delays=delays
    )
    delayed_test_features[story] = make_delayed(bert_test_feature[story], delays=delays)

In [13]:
# ndelays = 4
# delays = np.arange(1, ndelays + 1)

# del_training_stim = make_delayed(bert_train_feature_fast, delays)
# del_test_stim = make_delayed(bert_test_feature_fast, delays)

# fMRI data

In [41]:
train_fn = f"subject{subject}_reading_fmri_data_trn.hdf"
test_fn = f"subject{subject}_reading_fmri_data_val.hdf"

training_data = load_dict(os.path.join(reading_data_en_path, train_fn))
test_data = load_dict(os.path.join(reading_data_en_path, test_fn))

In [46]:
trim = 5
ztraining_data = np.vstack(
    [
        zscore(
            training_data[story][
                silence_length + noise_trim_length : -(noise_trim_length+silence_length), :
            ],
            axis=0,
        )
        for story in list(training_data.keys())
    ]
)
ztest_data = zscore(
    np.mean(test_data["story_11"], axis=0)[silence_length + noise_trim_length : -(noise_trim_length+silence_length), :], axis=0
)

In [47]:
ztraining_data.shape

(3737, 92970)

In [48]:
delayed_train_features[timescale].shape

(3737, 39936)

In [52]:
assert ztraining_data.shape[0] == delayed_train_features[timescale].shape[0]
assert ztest_data.shape[0] == delayed_test_features[timescale].shape[0]

# VEM

In [56]:
kfold = KFold(n_splits=5, shuffle=True, random_state=0)

alphas = np.logspace(1, 3, 10)

start = time.time()

reg = RidgeCV(alphas=alphas, cv=kfold.split(ztraining_data), store_cv_values=False)

reg.fit(np.nan_to_num(delayed_train_features[timescale]), np.nan_to_num(ztraining_data))

print(f"Training took {time.time() - start} seconds")